In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - MaMMUT

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_mammut.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mammut.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying MaMMUT to a Vertex AI Endpoint and making online predictions.

### Objective

- Deploy MaMMUT to a Vertex AI Endpoint.
- Make predictions to the endpoint including:
  - Answering questions about a given image.


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Setup Google Cloud project
# @markdown ### Prerequisites
# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. [Optional] [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git
! pip install -q gradio==4.21.0

import gradio as gr
import os
from datetime import datetime
from google.cloud import aiplatform
import importlib
from PIL import Image
from typing import List, Tuple

common_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.common_util"
)

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_URI = "gs://"  # @param {type: "string"}
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."

# Create a unique GCS bucket for this notebook, if not specified by the user.
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}"
    ! gsutil mb -l {REGION} {BUCKET_URI}
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
else:
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )

print(f"Using this GCS Bucket: {BUCKET_URI}")

! gcloud config set project $PROJECT_ID
! gcloud services enable language.googleapis.com

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")

# Set up default SERVICE_ACCOUNT
SERVICE_ACCOUNT = None
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print("Using this default Service Account:", SERVICE_ACCOUNT)

# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)


# The pre-built prediction docker image.
OPTIMIZED_TF_RUNTIME_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.nightly:latest"
)

def resize_image(image: Image.Image, new_width: int = 512) -> Image.Image:
  width, height = image.size
  print("original input image size:", width, ",", height)
  new_height = int(height * new_width / width)
  new_image = image.resize((new_width, new_height))
  return new_image

def deploy_mammut() -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
  """Deploy the model to a Vertex endpoint for prediction."""
  serving_env = {
    "MODEL_ID": "mammut",
    "DEPLOY_SOURCE": "notebook",
  }

  model_dir = "gs://mammut-unzip/mammut/checkpoints/cpu"

  upload_job_name = common_util.get_job_name_with_datetime(prefix="mammut-upload")

  model = aiplatform.Model.upload(
    display_name=upload_job_name,
    artifact_uri=model_dir,
    serving_container_image_uri=OPTIMIZED_TF_RUNTIME_IMAGE_URI,
    serving_container_args=[],
    location=REGION,
    serving_container_environment_variables=serving_env,
  )

  print("The uploaded model name is: ", upload_job_name)

  deploy_model_name = common_util.get_job_name_with_datetime(prefix="mammut-deploy")

  accelerator_type = "NVIDIA_TESLA_V100"
  accelerator_count = 1
  common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    is_for_training=False,
  )

  endpoint = model.deploy(
    deployed_model_display_name=deploy_model_name,
    traffic_split={"0": 100},
    machine_type="n1-standard-4",
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    min_replica_count=1,
    max_replica_count=1,
  )

  print("The deployed job name is: ", deploy_model_name)

  endpoint_id = endpoint.name
  print("endpoint id is: ", endpoint_id)
  return model, endpoint

def vqa_predict(
    endpoint: aiplatform.Endpoint,
    image: Image.Image,
    prompts: List[str],
    new_width: int = 1000,
) -> List[str]:
    """Predicts the answer to a question about an image using an Endpoint."""
    # Resize and convert image to base64 string.
    resized_image = resize_image(image, new_width)

    responses = []
    for question_prompt in prompts:
        if question_prompt:
          instances = [
    {
            "image_bytes":  {"b64": common_util.image_to_base64(resized_image)},
            "text": question_prompt
    }
          ]
          response = endpoint.predict(instances=instances)
          responses.append(response.predictions[0])
    return responses

## Run online prediction

Run online prediction with the TF SavedModel.

In [ ]:
# @title Deploy
# @markdown Upload TF SavedModel and deploy it to an endpoint for prediction. This step takes around 15 minutes to finish.

model, endpoint = deploy_mammut()

### Predict

The following section will use images from [pexels.com](https://www.pexels.com/) for demoing purposes. All the images have the following license: https://www.pexels.com/license/.

Images will be resized to a width of 1000 pixels by default since requests made to a Vertex Endpoint are limited to 1.500MB.

In [ ]:
# @title Visual Question Answering
# @markdown Use the deployed MaMMUT model to answer questions about a given image.

# @markdown ![](https://images.pexels.com/photos/4012966/pexels-photo-4012966.jpeg?w=1260&h=750)

# @markdown This can be either a Cloud Storage path (gs://\<image-path\>) or a public url (http://\<image-path\>)
image_url = "https://images.pexels.com/photos/4012966/pexels-photo-4012966.jpeg"  # @param {type:"string"}

if image_url.startswith("gs://"):
  local_image_path = "./images/test_image.jpg"
  common_util.download_gcs_file_to_local(image_url, local_image_path)
  image = common_util.load_img(local_image_path)
else:
  image = common_util.download_image(image_url)
display(image)

# @markdown You may leave question prompts empty and they will be ignored.
question_prompt_1 = "Is there a person in the image?"  # @param {type: "string"}
question_prompt_2 = "What is the person doing in the image?"  # @param {type: "string"}
question_prompt_3 = "What's the color of the cup?"  # @param {type: "string"}
question_prompt_4 = "How many laptops are in the image?"  # @param {type: "string"}

questions_list = [
    question_prompt_1,
    question_prompt_2,
    question_prompt_3,
    question_prompt_4,
]
questions_list = [question for question in questions_list if question]


answers = vqa_predict(endpoint, image, questions_list)

for question, answer in zip(questions_list, answers):
    print(f"Question: {question}")
    print(f"Answer: {answer}")

## Creating a webpage playground with Gradio

In [ ]:
# @title How to use

# @markdown **Prerequisites**
# @markdown -  Before you can upload an image to make a prediction, you need to select a Vertex prediction endpoint serving MaMMUT
# @markdown from the endpoint dropdown list that has been deployed in the current project and region.
# @markdown -  If no models have been deployed, you can create a new Vertex prediction
# @markdown endpoint by clicking "Deploy to Vertex" in the playground or running the `Deploy` cell above.
# @markdown   * New model deployment takes approximately 15 minutes. You can check the progress at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).

# @markdown **How to use**

# @markdown Just run this cell and a link to the playground formatted as `https://####.gradio.live` will be outputted.
# @markdown This link will take you to the playground in a separate browser tab.

def list_mammut_endpoints() -> list[str]:
    """Returns all valid prediction endpoints for in the project and region."""
    # Gets all the valid endpoints in the project and region.
    endpoints = aiplatform.Endpoint.list(order_by="create_time desc")
    # Filters out the endpoints which do not have a deployed model, and the endpoint is for image generation
    endpoints = list(
        filter(
            lambda endpoint: endpoint.traffic_split
            and "mammut" in endpoint.display_name.lower(),
            endpoints,
        )
    )

    endpoint_names = list(
        map(
            lambda endpoint: f"{endpoint.name} - {endpoint.display_name[:40]}",
            endpoints,
        )
    )

    if not endpoint_names:
        gr.Warning(
            "No prediction endpoints were found. Create an Endpoint first."
        )

    return endpoint_names

def deploy_model_handler(model_choice: str) -> None:
    gr.Info("Starting model deployment.")
    model, endpoint = deploy_mammut()
    gr.Info(f"Deploying model ID: {model.name}, endpoint ID: {endpoint.name}")

def get_endpoint(endpoint_name: str) -> aiplatform.Endpoint:
    """Returns a Vertex endpoint for the given endpoint_name."""
    endpoint_id = endpoint_name.split(" - ")[0]
    endpoint = aiplatform.Endpoint(
        f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_id}"
    )
    return endpoint

def predict_handler(
    endpoint_name: str,
    image: Image.Image,
    prompt: str,
) -> str:
    if not endpoint_name:
        raise gr.Error("Select (or deploy) a model first!")
    if not image:
        raise gr.Error("You must upload an image!")
    endpoint = get_endpoint(endpoint_name)
    return vqa_predict(endpoint, image, [prompt])[0]


tip_text = r"""
<b> Tips: </b>
1. Select a Vertex prediction endpoint with a deployed MaMMUT model or click `Deploy to Vertex` to deploy MaMMUT to Vertex.
2. New model deployment takes approximately 15 minutes. You can check the progress at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).
3. After the model deployment is complete, click `Refresh Endpoints list` to view the new endpoint in the dropdown list.
"""

css = """
.gradio-container {
  width: 85% !important
}
"""
with gr.Blocks(
    css=css, theme=gr.themes.Default(primary_hue="orange", secondary_hue="blue")
) as demo:
    gr.Markdown("# Model Garden Playground for MaMMUT")
    with gr.Row(equal_height=True):
        with gr.Column(scale=3):
            gr.Markdown(tip_text)
        with gr.Column(scale=2):
            with gr.Row():
                endpoint_name = gr.Dropdown(
                    scale=7,
                    label="Select a model previously deployed on Vertex",
                    choices=list_mammut_endpoints(),
                    value=None,
                )
                refresh_button = gr.Button(
                    "Refresh Endpoints list",
                    scale=1,
                    variant="primary",
                    min_width=10,
                )
            with gr.Row():
                deploy_model_button = gr.Button(
                    "Deploy a new model",
                    scale=1,
                    variant="primary",
                    min_width=10,
                )
    with gr.Row(equal_height=True):
        with gr.Column(scale=1):
            image_input = gr.Image(
                show_label=True,
                type="pil",
                label="Upload",
                visible=True,
                height=400,
            )
            with gr.Group():
              text_input_box = gr.Textbox(label="Question", lines=1)
              submit_button = gr.Button("Answer", variant="primary")
        with gr.Column(scale=1):
            image_output = gr.Image(label="Image response:", visible=False)
            text_output = gr.Textbox(label="Text response:")

    refresh_button.click(
        fn=lambda: gr.update(choices=list_mammut_endpoints()),
        outputs=[endpoint_name],
    )
    deploy_model_button.click(
        deploy_model_handler,
        outputs=[],
    )
    submit_button.click(
        fn=predict_handler,
        inputs=[
            endpoint_name,
            image_input,
            text_input_box,
        ],
        outputs=[text_output],
    )
show_debug_logs = True  # @param {type: "boolean"}
demo.queue()
demo.launch(share=True, inline=False, inbrowser=True, debug=show_debug_logs, show_error=True)

## Clean up resources


In [ ]:
# @title Run

# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Delete endpoint resource.
endpoint.delete(force=True)

# Delete model resource.
model.delete()

# Delete Cloud Storage objects that were created.
delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI